In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import glob
import argparse
import os
import argparse
import numpy as np
import pickle
from PIL import Image
from tqdm import tqdm
from torch import nn

In [2]:
cd ..

E:\CVprojects\Butterflies


In [3]:
# !git clone https://github.com/orobix/Prototypical-Networks-for-Few-shot-Learning-PyTorch.git

In [4]:
def load_dict(filename_):
    with open(filename_, 'rb') as f:
        ret_di = pickle.load(f)
    return ret_di
def save_dict(di_, filename_):
    with open(filename_, 'wb') as f:
        pickle.dump(di_, f)
def init_optim(opt, model):
    '''
    Initialize optimizer
    '''
    params = [param for param in model.parameters() if param.requires_grad]
    return torch.optim.Adam(params=params,
                            lr=opt.learning_rate)

In [135]:
class Butterfly200DataSet_sub(Dataset):
    def __init__(self,split_dict_path,precashe=True,mode='train'):
        super(Butterfly200DataSet_sub, self).__init__()
        split_dict = load_dict(split_dict_path)
        self.paths = split_dict[mode]
        self.images_classes = {}
        self.y = []


        for path in self.paths:
            cls = int(os.path.basename(path.split('.')[0]))-1
            if cls not in self.images_classes:
                self.images_classes[cls] = []
            self.images_classes[cls].append(path)
            self.y.append(cls)
#         np.random.shuffle(self.y)
        classes = np.unique(self.y)
        self.map = {i:cls for i,cls in enumerate(classes)}
        map2 = {cls:i for i,cls in enumerate(classes)}
        self.y = [map2[e] for e in self.y]
        self.imgs_cashe = {}
        if precashe:
            for i in tqdm(range(len(self.paths))):
                img = self.load_img(self.paths[i])
                label = self.y[i]
                self.imgs_cashe[i] = (img,label)
    def load_img(self,path):
        x = torch.load(path)
        return x
    def __getitem__(self, index):
        if  index in self.imgs_cashe:
            return self.imgs_cashe[index][0].cuda(non_blocking=True),self.imgs_cashe[index][1]
        else:
            img = self.load_img(self.paths[index])
            img = img.cuda(non_blocking=True)
            label = self.y[index]
        return img,label
    def __len__(self):
        return len(self.paths)


In [136]:
cd E:\CVprojects\Butterflies

E:\CVprojects\Butterflies


In [137]:
split_dict_name = "split_dict_sub.pkl"
split_dict_path = os.path.join(os.getcwd(),split_dict_name)
split_dict_path

'E:\\CVprojects\\Butterflies\\split_dict_sub.pkl'

In [138]:
train_dataset = Butterfly200DataSet_sub(split_dict_path=split_dict_path,mode='train')
len(train_dataset)

100%|████████████████████████████████████████████████████████████████████████████| 1726/1726 [00:01<00:00, 1693.82it/s]


1726

In [139]:
train_dataset.map

{0: 45, 1: 55, 2: 60, 3: 121}

In [140]:
val_dataset = Butterfly200DataSet_sub(split_dict_path=split_dict_path,mode='val')
# test_dataset = Butterfly200DataSet(split_dict_path=split_dict_path,mode='val')

len(val_dataset)

100%|██████████████████████████████████████████████████████████████████████████████| 213/213 [00:00<00:00, 1365.39it/s]


213

In [141]:
def train(opt, tr_dataloader, model, optim, val_dataloader=None):
    '''
    Train the model with the prototypical learning algorithm
    '''

    device = 'cuda:0' if torch.cuda.is_available() and opt.cuda else 'cpu'

    if val_dataloader is None:
        best_state = None
    train_loss = []
    train_acc = []
    val_loss = []
    val_acc = []
    best_acc = 0

    best_model_path = os.path.join(opt.experiment_root, 'best_model.pth')
    last_model_path = os.path.join(opt.experiment_root, 'last_model.pth')
    EntroLoss = nn.CrossEntropyLoss()
    for epoch in range(opt.epochs):
        print('=== Epoch: {} ==='.format(epoch))
        tr_iter = iter(tr_dataloader)
        model.train()
        
        for batch in tqdm(tr_iter):
            optim.zero_grad()
            x, y = batch
            x, y = x.to(device), y.to(device)
            model_output = model(x)
#             loss, acc = loss_fn(model_output, target=y,
#                                 n_support=opt.num_support_tr)
            model_output = model_output.to('cpu')
            y = y.to('cpu')
            loss =EntroLoss(model_output, y)
            acc = torch.sum(torch.argmax(model_output,axis=1) == y).item()/y.shape[0]
            loss.backward()
            optim.step()
            train_loss.append(loss.item())
            train_acc.append(acc)
            torch.cuda.empty_cache()
        avg_loss = np.mean(train_loss[-opt.batch_size:])
        avg_acc = np.mean(train_acc[-opt.batch_size:])
        print('Avg Train Loss: {}, Avg Train Acc: {}'.format(avg_loss, avg_acc))
#         lr_scheduler.step()
        if val_dataloader is None:
            continue
        val_iter = iter(val_dataloader)
        model.eval()
        for batch in tqdm(val_iter):
            x, y = batch
            x, y = x.to(device), y.to(device)
            model_output = model(x)
            y = y.to('cpu')
            model_output = model_output.to('cpu')
            loss =EntroLoss(model_output, y)
            acc = torch.sum(torch.argmax(model_output,axis=1) == y).item()/y.shape[0]
            
            val_loss.append(loss.item())
            val_acc.append(acc)
            torch.cuda.empty_cache()
        avg_loss = np.mean(val_loss[-opt.batch_size:])
        avg_acc = np.mean(val_acc[-opt.batch_size:])
        postfix = ' (Best)' if avg_acc >= best_acc else ' (Best: {})'.format(
            best_acc)
        print('Avg Val Loss: {}, Avg Val Acc: {}{}'.format(
            avg_loss, avg_acc, postfix))
        if avg_acc >= best_acc:
            torch.save(model.state_dict(), best_model_path)
            best_acc = avg_acc
            best_state = model.state_dict()

    torch.save(model.state_dict(), last_model_path)

    for name in ['train_loss', 'train_acc', 'val_loss', 'val_acc']:
        save_list_to_file(os.path.join(opt.experiment_root,
                                       name + '.txt'), locals()[name])

    return best_state, best_acc, train_loss, train_acc, val_loss, val_acc


def test(opt, test_dataloader, model):
    '''
    Test the model trained with the prototypical learning algorithm
    '''
    device = 'cuda:0' if torch.cuda.is_available() and opt.cuda else 'cpu'
    avg_acc = list()
    for epoch in tqdm(range(opt.test_epochs)):
        test_iter = iter(test_dataloader)
        for batch in test_iter:
            x, y = batch
            x, y = x.to(device), y.to(device)
            model_output = model(x)
            _, acc = loss_fn(model_output, target=y,
                             n_support=opt.num_support_val)
            avg_acc.append(acc.item())
    avg_acc = np.mean(avg_acc)
    print('Test Acc: {}'.format(avg_acc))

    return avg_acc







In [164]:
d = {
    'experiment_root':'base_exp',
     'epochs':100,
     'cuda':True,
    'learning_rate':0.000001,
    'batch_size':16,
     'check_point_path':'base_exp/best_model.pth'
     }
options = argparse.ArgumentParser()
for key in d:
    options.add_argument('-'+key, '--'+key, type=type(d[key]))
for key in d:
    setattr(options, key, d[key])
#     options.parse_args(['-'+key, str(d[key])])


In [165]:
options.learning_rate

1e-06

In [166]:
'''
Initialize everything and train
'''
if not os.path.exists(options.experiment_root):
    os.makedirs(options.experiment_root)

if torch.cuda.is_available() and not options.cuda:
    print("WARNING: You have a CUDA device, so you should probably run with --cuda")


tr_dataloader = DataLoader(train_dataset,shuffle=True,batch_size=options.batch_size)
val_dataloader = DataLoader(val_dataset,shuffle=False,batch_size=options.batch_size)



In [167]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
model.fc = nn.Linear(in_features=512, out_features=4, bias=True)
# if  options.check_point_path!='':
#     model.load_state_dict(torch.load(options.check_point_path))



Using cache found in C:\Users\EMT/.cache\torch\hub\pytorch_vision_v0.10.0


In [168]:
options.check_point_path

'base_exp/best_model.pth'

In [169]:
model = model.to('cuda')

In [170]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [171]:
# model.requires_grad = False
# for name, W in model.named_parameters():
#     if 'fc' in name: #or ('layer4' in name and 'conv2' in name):
#         print(name)
#         W.requires_grad = True
#     else:
#         W.requires_grad = False
# model.layer4[-1].conv2.requires_grad = True

In [172]:
for name, W in model.named_parameters():
    if W.requires_grad:
        print(name,W.requires_grad)

conv1.weight True
bn1.weight True
bn1.bias True
layer1.0.conv1.weight True
layer1.0.bn1.weight True
layer1.0.bn1.bias True
layer1.0.conv2.weight True
layer1.0.bn2.weight True
layer1.0.bn2.bias True
layer1.1.conv1.weight True
layer1.1.bn1.weight True
layer1.1.bn1.bias True
layer1.1.conv2.weight True
layer1.1.bn2.weight True
layer1.1.bn2.bias True
layer2.0.conv1.weight True
layer2.0.bn1.weight True
layer2.0.bn1.bias True
layer2.0.conv2.weight True
layer2.0.bn2.weight True
layer2.0.bn2.bias True
layer2.0.downsample.0.weight True
layer2.0.downsample.1.weight True
layer2.0.downsample.1.bias True
layer2.1.conv1.weight True
layer2.1.bn1.weight True
layer2.1.bn1.bias True
layer2.1.conv2.weight True
layer2.1.bn2.weight True
layer2.1.bn2.bias True
layer3.0.conv1.weight True
layer3.0.bn1.weight True
layer3.0.bn1.bias True
layer3.0.conv2.weight True
layer3.0.bn2.weight True
layer3.0.bn2.bias True
layer3.0.downsample.0.weight True
layer3.0.downsample.1.weight True
layer3.0.downsample.1.bias True
la

In [173]:
optim = init_optim(options, model)
# lr_scheduler = init_lr_scheduler(options, optim)

In [174]:
loader = iter(tr_dataloader)


In [175]:
ex = next(loader)

In [176]:
ex[0].shape

torch.Size([16, 3, 224, 224])

In [177]:
out = model(ex[0])

In [178]:
out.shape

torch.Size([16, 4])

In [179]:
loss = nn.CrossEntropyLoss()
loss(out.to('cpu'), ex[1])

tensor(1.3649, grad_fn=<NllLossBackward0>)

In [180]:
m = nn.Softmax(dim=1)

In [181]:
torch.argmax(out.to('cpu'),axis=1).shape

torch.Size([16])

In [182]:
torch.sum(torch.argmax(out.to('cpu'),axis=1) == ex[1]).item()/ex[1].shape[0]

0.375

In [183]:
ex[1].shape[0]

16

In [184]:
ex[1].device

device(type='cpu')

In [ ]:
res = train(opt=options,
            tr_dataloader=tr_dataloader,
            val_dataloader=val_dataloader,
            model=model,
            optim=optim)
best_state, best_acc, train_loss, train_acc, val_loss, val_acc = res

=== Epoch: 0 ===


100%|████████████████████████████████████████████████████████████████████████████████| 108/108 [00:09<00:00, 10.93it/s]


Avg Train Loss: 1.3277250081300735, Avg Train Acc: 0.4380580357142857


100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:00<00:00, 33.02it/s]


Avg Val Loss: 1.3232079318591528, Avg Val Acc: 0.4035714285714286 (Best)
=== Epoch: 1 ===


...

100%|████████████████████████████████████████████████████████████████████████████████| 108/108 [00:59<00:00,  1.80it/s]


Avg Train Loss: 0.12744149169884622, Avg Train Acc: 0.9765625


100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:02<00:00,  6.10it/s]


Avg Val Loss: 0.14281172247137874, Avg Val Acc: 0.9609375 (Best: 0.96875)
=== Epoch: 34 ===


 72%|██████████████████████████████████████████████████████████▌                      | 78/108 [00:40<00:14,  2.11it/s]

In [ ]:
model.load_state_dict(torch.load('base_exp/best_model.pth'))

In [214]:
loader = iter(val_dataloader)
ex = next(loader)




In [215]:
torch.argmax(model(ex[0]),1).to('cpu')==ex[1].to('cpu')

tensor([ True, False,  True,  True, False,  True,  True,  True, False, False,
        False, False, False, False, False,  True])

In [216]:
ex[1]

tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [68]:
print('Testing with last model..')
test(opt=options,
      test_dataloader=test_dataloader,
      model=model)

Testing with last model..
Test Acc: 0.44860000163316727


0.44860000163316727

In [ ]:
model.load_state_dict(best_state)
print('Testing with best model..')
test(opt=options,
      test_dataloader=test_dataloader,
      model=model)